# সেশন ১ – চ্যাট বুটস্ট্র্যাপ (ফাউন্ড্রি লোকাল)

এই নোটবুকটি ফাউন্ড্রি লোকাল বুটস্ট্র্যাপ করে, পছন্দের মডেল এলিয়াস ডাউনলোড করে এবং একটি স্ট্যান্ডার্ড ও স্ট্রিমিং চ্যাট সম্পন্ন করে।


# পরিস্থিতি
এই সেশনে Foundry Local ব্যবহার করে একটি ছোট ভাষার মডেলকে সাড়া দেওয়ার জন্য প্রয়োজনীয় ন্যূনতম বিষয়গুলো তুলে ধরা হয়েছে। আপনি যা করবেন:
- SDK / ক্লায়েন্ট ডিপেনডেন্সি ইনস্টল করবেন।
- একটি নির্ধারিত উপনাম (ডিফল্ট: `phi-4-mini`) এর জন্য Foundry Local ম্যানেজার ইনিশিয়ালাইজ করবেন।
- মডেলের মেটাডেটায় ঐচ্ছিক ক্ষেত্রগুলোর সহনশীলতার জন্য একটি প্রতিরক্ষামূলক মাঙ্কি-প্যাচ প্রয়োগ করবেন।
- একটি স্ট্যান্ডার্ড চ্যাট কমপ্লিশন অনুরোধ পাঠাবেন।
- প্রতিক্রিয়া টোকেন-বাই-টোকেন স্ট্রিম করবেন।

লক্ষ্য হলো RAG, রাউটিং বা এজেন্টে যাওয়ার আগে আপনার লোকাল রানটাইম এবং নেটওয়ার্ক পথ যাচাই করা।


### ব্যাখ্যা: নির্ভরতা ইনস্টলেশন
এই ক্ষুদ্র চ্যাট ফ্লো চালানোর জন্য প্রয়োজনীয় পাইথন প্যাকেজগুলো ইনস্টল করা হয়:
- `foundry-local-sdk`: স্থানীয় মডেল এবং সার্ভিস লাইফসাইকেল পরিচালনা করার জন্য।
- `openai`: চ্যাট সম্পন্ন করার জন্য পরিচিত ক্লায়েন্ট অ্যাবস্ট্রাকশন।
- `rich`: নোটবুক আউটপুটকে আরও পরিষ্কারভাবে প্রদর্শনের জন্য সুন্দর প্রিন্টিং।

পুনরায় চালানো নিরাপদ (আইডেমপোটেন্ট)। যদি আপনার পরিবেশে ইতোমধ্যেই এগুলো থাকে, তাহলে এটি এড়িয়ে যেতে পারেন।


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### ব্যাখ্যা: মূল আমদানি
নোটবুক জুড়ে ব্যবহৃত মডিউলগুলি নিয়ে আসে:
- `FoundryLocalManager` স্থানীয় মডেল রানটাইমের সাথে ইন্টারফেস করার জন্য।
- `OpenAI` ক্লায়েন্ট যাতে আমরা পরিচিত চ্যাট সম্পূর্ণ API পৃষ্ঠাকে পুনরায় ব্যবহার করতে পারি।
- `rich.print` স্টাইলযুক্ত আউটপুটের জন্য।

এখানে কোনো নেটওয়ার্ক কল হয় না—এটি শুধুমাত্র নেমস্পেস প্রস্তুত করে।


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### ব্যাখ্যা: ম্যানেজার ইনিশিয়ালাইজেশন এবং মেটাডেটা প্যাচ
নির্বাচিত এলিয়াসের জন্য `FoundryLocalManager` ইনিশিয়ালাইজ করে এবং একটি প্রতিরক্ষামূলক মাঙ্কি-প্যাচ প্রয়োগ করে যাতে `promptTemplate` যদি `null` হয়, তবে পরিষেবার প্রতিক্রিয়া সুন্দরভাবে পরিচালনা করা যায়।

মূল ফলাফল:
- পরিষেবার অবস্থা এবং এন্ডপয়েন্ট নিশ্চিত করে।
- ক্যাশ করা মডেলগুলোর তালিকা দেখায় (স্থানীয় স্টোর যাচাই করে)।
- এলিয়াসের জন্য নির্দিষ্ট মডেল আইডি সমাধান করে (পরবর্তী চ্যাট কলগুলিতে ব্যবহৃত হয়)।

যদি কাঁচা পরিষেবা মেটাডেটায় যাচাইকরণের সমস্যা দেখা দেয়, এই প্যাটার্নটি দেখায় কীভাবে SDK-কে ফর্ক না করেই তা পরিষ্কার করা যায়।


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### ব্যাখ্যা: বেসিক চ্যাট সম্পূর্ণতা
স্থানীয় Foundry এন্ডপয়েন্টে নির্দেশিত একটি `OpenAI`-সামঞ্জস্যপূর্ণ ক্লায়েন্ট তৈরি করে এবং একটি একক নন-স্ট্রিমিং চ্যাট সম্পূর্ণতা সম্পাদন করে। এখানে ফোকাস করুন:
- নিশ্চিত করুন যে মডেলটি ত্রুটি ছাড়াই সাড়া দেয়।
- লেটেন্সি / আউটপুট ফরম্যাট যাচাই করুন।
- সংস্থান সংরক্ষণের জন্য `max_tokens` সীমিত রাখুন।

যদি এটি ব্যর্থ হয়, তাহলে পুনরায় পরীক্ষা করুন যে Foundry Local সার্ভিসটি চালু আছে এবং এলিয়াসটি সঠিকভাবে রেজলভ করছে।


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### ব্যাখ্যা: স্ট্রিমিং চ্যাট সম্পূর্ণতা
উন্নত অনুভূত লেটেন্সি এবং ইন্টারঅ্যাকটিভ UX এর জন্য টোকেন স্ট্রিমিং প্রদর্শন করে। লুপটি ইনক্রিমেন্টাল ডেল্টাস প্রিন্ট করে যখন তারা আসে:
- চ্যাট UI-গুলোর জন্য উপযোগী যেখানে প্রাথমিক আংশিক আউটপুট গুরুত্বপূর্ণ।
- আপনাকে টোকেন থ্রুপুট বনাম সম্পূর্ণ সম্পূর্ণতার লেটেন্সি পরিমাপ করতে দেয়।

আপনি এই প্যাটার্নটি টোকেন জমা করতে, একটি প্রগ্রেস উইজেট আপডেট করতে, বা মাঝপথে জেনারেশন বন্ধ করতে মানিয়ে নিতে পারেন।


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
